In [ ]:
from tunable_agents import utility, agent
import utils
import gin
import numpy as np
import os
import tensorflow as tf
from tensorflow import keras
from tunable_agents.environments.gathering_env import gathering_env
import matplotlib.pyplot as plt

%matplotlib inline
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
#gpus = tf.config.experimental.list_physical_devices('GPU')
#tf.config.experimental.set_memory_growth(gpus[0], True)

In [ ]:
configs_dir = "C:/Users/maler/Google Drive/Personale/Università/Master/Tesi/Code/tunable-agents-MORL/configs/"

gin_files = [configs_dir + "envs/cumulative_linear_dual_threshold_env.gin", configs_dir + "qnets/128_128_64_model.gin"]
#gin_files = [configs_dir + "envs/fixed_env.gin", configs_dir + "qnets/128_128_64_model.gin"]
gin_bindings = []
utility.load_gin_configs(gin_files, gin_bindings)

In [ ]:
root_path = "C:/Users/maler/Federico/Università/Master/Tesi/experiments_results/"
root_path = "C:/Users/maler/Federico/Università/Master/Tesi/new_results/"
experiment = "128_128_64_model-cum_linear_dual_threshold_env-replication-1"

experiment_dir = os.path.join(root_path, experiment)
model_dir = os.path.join(experiment_dir, 'model')
model_path = os.path.join(model_dir, 'dqn_model.h5')


env = utility.create_environment()
tf_agent = agent.DQNAgent(epsilon=lambda: 0.1, obs_spec=env.observation_spec())

#tf_agent.load_model(model_path)


In [ ]:
time_step = env.reset()

In [ ]:
action = tf_agent.greedy_policy(time_step.observation)
action = 2
time_step = env.step(action)

In [ ]:
plt.figure(figsize=(15,7), dpi=100)
image = time_step.observation['state_obs'][:, :, 6:]
plt.imshow(image)
plt.title('Utility: {}    Action: {}'.format(time_step.reward, action))
plt.axis('off')
plt.show()

if time_step.is_last():
    print('END EPISODE')

In [ ]:
print(env._utility_func.utility_repr)
print(env._cumulative_rewards)
print(env._prev_step_utility)
print(env._interests)

# Agent training

In [ ]:
def collection_step(env, tf_agent: agent.DQNAgent,
                    replay_memory: agent.ReplayMemory, reward_tracker: agent.RewardTracker,
                    collect_episodes: int) -> None:
    """Samples transitions with the given Driver."""
    if not collect_episodes:
        return

    for _ in range(collect_episodes):
        # Reset env
        ts = env.reset()
        observations = ts.observation

        episode_reward = 0
        done = False
        while not done:
            action = tf_agent.epsilon_greedy_policy(observations, training=True)
            ts = env.step(action)
            next_obs, reward, done = ts.observation, ts.reward, ts.is_last()

            replay_memory.append((observations, action, reward, next_obs, done))
            observations = next_obs

            episode_reward += reward

        reward_tracker.append(episode_reward)

    return episode_reward


In [ ]:
replay_memory = agent.ReplayMemory(maxlen=6000)
reward_tracker = agent.RewardTracker(maxlen=100)

collection_step(env, tf_agent, replay_memory, reward_tracker, collect_episodes=100)


In [ ]:
experiences = replay_memory.sample(32)
states, actions, rewards, next_states, dones = experiences

In [ ]:
next_Q_values = tf_agent._target_model(
    [next_states["state_obs"]] +
    [next_states[key] for key in sorted(next_states.keys() - ["state_obs"])])

max_next_Q_values = np.max(next_Q_values, axis=1)
target_Q_values = (rewards + (1 - dones) * tf_agent._gamma * max_next_Q_values)
target_Q_values = target_Q_values.reshape(-1, 1)  # Make column vector

# Mask to only consider action taken
mask = tf.one_hot(actions, tf_agent._output_size)  #pylint: disable=no-value-for-parameter
# Compute loss and gradient for predictions on 'states'
with tf.GradientTape() as tape:
    all_Q_values = tf_agent._model([states["state_obs"]] +
                                [states[key] for key in sorted(states.keys() - ["state_obs"])],
                                training=True)
    Q_values = tf.reduce_sum(all_Q_values * mask, axis=1, keepdims=True)
    loss = tf.reduce_mean(tf_agent._loss_fn(target_Q_values, Q_values))
grads = tape.gradient(loss, tf_agent._model.trainable_variables)

In [ ]:
print(grads[0].shape)

In [ ]:
print(actions)
print(rewards)
print(next_Q_values.numpy())